In [1]:
import nivapy3 as nivapy
import xarray as xr

# Estimating mean runoff for the 1000 Lakes catchments

Calculate mean runoff (1991-2020) for each of the 1000 Lakes catchments using NVE's new "runoff normal" dataset, which was released in autumn 2022. Øyvind would like two versions of the data:

 1. Mean values for the catchment upstream of the monitoring point
 
 2. The value for grid cell containing the sampling point

## 1. Get 1000 Lakes catchment polygons

Originally derived by Jose, these are available on the Hub's PostGIS database.

In [2]:
# Find 1000 Lakes project in Hub's database
eng = nivapy.da.connect_postgis()
nivapy.da.select_jhub_projects(eng)

Connection successful.


,project_id,project_code,project_name,aquamonitor_id,contact,description
0,1,Refelv_Over,Overvåking av referanseelver,11226.0,None,Overvåking av referanseelver
1,2,190246,CL Vestland,NaN,KAU,Critical loads calculations for Vestland
2,4,O-190091,Nasjonal Innsjøundersøkelse 2019,12433.0,HIN,1000 Lakes survey in 2019


In [3]:
# Get outflows and catchments. We want project_id = 4
stn_gdf, cat_gdf = nivapy.da.select_jhub_project_catchments([4], eng)
assert len(stn_gdf) == len(cat_gdf)
print(len(cat_gdf), "catchment polygons.")
del cat_gdf["station_id"]

# Add area in km2
cat_gdf["area_km2"] = cat_gdf.to_crs({"proj": "cea"})["geom"].area / 1e6

cat_gdf.head()

1001 catchment polygons.


,station_code,station_name,aquamonitor_id,longitude,latitude,geom,area_km2
0,221-1-2,Langtjern,26070,11.850610,59.809915,"MULTIPOLYGON (((11.85402 59.81418, 11.85437 59...",0.426415
1,101-2-7,Hokksjøen,26071,11.559946,59.005125,"MULTIPOLYGON (((11.59333 59.01263, 11.59351 59...",5.500691
2,402-2-13,Sætertjern,26072,12.443670,60.057771,"MULTIPOLYGON (((12.43744 60.07300, 12.43815 60...",1.572078
3,419-1-25,Mjøgsjøen,26073,11.845276,60.332035,"MULTIPOLYGON (((11.83902 60.33438, 11.84010 60...",1.018257
4,425-2-2,Kottern,26074,12.511994,60.587319,"MULTIPOLYGON (((12.51086 60.59991, 12.51195 60...",2.126635


## 2. Zonal statistics

In [5]:
%%time

nve_runoff_path = (
    r"/home/jovyan/shared/teotil3/nve_hbv_data/runoff_1991-2020_mm_year.tif"
)
gdf = nivapy.spatial.zonal_statistics(
    cat_gdf, nve_runoff_path, "runoff_mm/yr", all_touched=True
)

# Round values
for col in gdf.columns:
    if col.endswith("_km2"):
        gdf[col] = gdf[col].round(4)
    elif col.endswith("_mm/yr"):
        gdf[col] = gdf[col].round(0)

display(gdf.describe())
display(gdf.head())

# Save dataframe
cols = [col for col in gdf.columns if col != gdf.geometry.name]
df = gdf[cols]
csv_path = "../data/1000_lakes_catchment_mean_runoff_1991-2020.csv"
df.to_csv(csv_path, index=False)

,aquamonitor_id,longitude,latitude,area_km2,min_runoff_mm/yr,max_runoff_mm/yr,mean_runoff_mm/yr,count_runoff_mm/yr,std_runoff_mm/yr,median_runoff_mm/yr,percentile_25_runoff_mm/yr,percentile_75_runoff_mm/yr
count,1001.000000,1001.000000,1001.000000,1001.000000,1000.000000,1000.000000,1000.000000,1001.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,27878.681319,11.145738,62.364697,77.936084,1331.939000,1591.263000,1450.956000,92.046953,71.072000,1447.765000,1401.858000,1497.77700
std,6867.542830,6.061257,3.739985,689.524993,857.077917,989.421718,901.426328,727.803304,82.782424,899.051363,877.861942,928.03581
min,26070.000000,4.695709,58.072318,0.106100,32.000000,333.000000,304.000000,0.000000,0.000000,299.000000,298.000000,311.00000
25%,26335.000000,7.142665,59.567290,0.918600,618.750000,804.000000,717.500000,4.000000,22.000000,716.500000,687.750000,751.00000
50%,26606.000000,8.926264,60.951266,2.677800,1153.000000,1373.000000,1255.500000,9.000000,44.000000,1247.000000,1214.000000,1296.00000
75%,26859.000000,12.162593,64.082348,8.909300,1831.500000,2171.500000,1997.750000,18.000000,89.000000,1994.000000,1928.500000,2050.00000
max,71862.000000,30.653056,71.063386,16529.795000,4399.000000,5530.000000,4805.000000,17303.000000,802.000000,4921.000000,4479.000000,5318.00000


,station_code,station_name,aquamonitor_id,longitude,latitude,area_km2,min_runoff_mm/yr,max_runoff_mm/yr,mean_runoff_mm/yr,count_runoff_mm/yr,std_runoff_mm/yr,median_runoff_mm/yr,percentile_25_runoff_mm/yr,percentile_75_runoff_mm/yr,geom
0,221-1-2,Langtjern,26070,11.850610,59.809915,0.4264,518.0,532.0,525.0,4,5.0,525.0,521.0,529.0,"MULTIPOLYGON (((11.85402 59.81418, 11.85437 59..."
1,101-2-7,Hokksjøen,26071,11.559946,59.005125,5.5007,610.0,703.0,667.0,14,26.0,668.0,654.0,688.0,"MULTIPOLYGON (((11.59333 59.01263, 11.59351 59..."
2,402-2-13,Sætertjern,26072,12.443670,60.057771,1.5721,454.0,531.0,493.0,6,27.0,493.0,471.0,515.0,"MULTIPOLYGON (((12.43744 60.07300, 12.43815 60..."
3,419-1-25,Mjøgsjøen,26073,11.845276,60.332035,1.0183,479.0,509.0,492.0,3,13.0,488.0,483.0,499.0,"MULTIPOLYGON (((11.83902 60.33438, 11.84010 60..."
4,425-2-2,Kottern,26074,12.511994,60.587319,2.1266,465.0,558.0,497.0,7,29.0,494.0,476.0,506.0,"MULTIPOLYGON (((12.51086 60.59991, 12.51195 60..."


CPU times: user 14.3 s, sys: 842 ms, total: 15.1 s
Wall time: 15.5 s


There is a problem for one site, which is located on a island not covered by the NVE dataset. This catchment is very small, so it is most easily filled manually.

In [6]:
gdf.query("`count_runoff_mm/yr` == 0")

,station_code,station_name,aquamonitor_id,longitude,latitude,area_km2,min_runoff_mm/yr,max_runoff_mm/yr,mean_runoff_mm/yr,count_runoff_mm/yr,std_runoff_mm/yr,median_runoff_mm/yr,percentile_25_runoff_mm/yr,percentile_75_runoff_mm/yr,geom
774,1866-1-2,Ulvøyvatn,26883,15.028698,68.35503,0.2953,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((15.02903 68.35419, 15.02952 68..."


Using ArcGIS, the runoff values for this catchment are:

    Count: 1
    Std: 0
    All other stats: 1869 mm/yr
    
## 3. Values for sampling locations

In [7]:
stn_gdf

,station_id,station_code,station_name,aquamonitor_id,longitude,latitude,geom
0,259,221-1-2,Langtjern,26070,11.850610,59.809915,POINT (11.85061 59.80992)
1,260,101-2-7,Hokksjøen,26071,11.559946,59.005125,POINT (11.55995 59.00512)
2,261,402-2-13,Sætertjern,26072,12.443670,60.057771,POINT (12.44367 60.05777)
3,262,419-1-25,Mjøgsjøen,26073,11.845276,60.332035,POINT (11.84528 60.33203)
4,263,425-2-2,Kottern,26074,12.511994,60.587319,POINT (12.51199 60.58732)
...,...,...,...,...,...,...,...
996,1255,428-2-202,Gløtsjøen,71718,12.119557,61.538025,POINT (12.11956 61.53802)
997,1256,623-603,Breidlivatnet,71719,10.149576,59.974952,POINT (10.14958 59.97495)
998,1257,938-3-203,Gyvatn,71723,7.682361,58.704930,POINT (7.68236 58.70493)
999,1258,1432-2-205,Femtevatnet,71725,6.611959,61.473289,POINT (6.61196 61.47329)


In [8]:
# Get stn x and y in grid co-ords (EPSG 25833)
crs = "epsg:25833"
stn_gdf["x"] = stn_gdf.to_crs(crs)["geom"].x
stn_gdf["y"] = stn_gdf.to_crs(crs)["geom"].y

# Open runoff data with xarray
ds = xr.open_dataset(nve_runoff_path)

# Interpolate no data
ds = ds.rio.interpolate_na(method="nearest")

# Extract values for points
x_indexer = xr.DataArray(
    stn_gdf["x"], dims=["aquamonitor_id"], coords=[stn_gdf["aquamonitor_id"]]
)
y_indexer = xr.DataArray(
    stn_gdf["y"], dims=["aquamonitor_id"], coords=[stn_gdf["aquamonitor_id"]]
)
pts_ds = ds.sel(x=x_indexer, y=y_indexer, method="nearest")
pts_df = pts_ds.to_dataframe().reset_index()
pts_df = pts_df[["aquamonitor_id", "band_data"]]
pts_df.columns = ["aquamonitor_id", "runoff_mm/yr"]
pts_df["runoff_mm/yr"] = pts_df["runoff_mm/yr"].round(0).astype(int)

csv_path = "../data/1000_lakes_sample_point_runoff_1991-2020.csv"
pts_df.to_csv(csv_path, index=False)

pts_df.describe()

,aquamonitor_id,runoff_mm/yr
count,1001.000000,1001.000000
mean,27878.681319,1419.823177
std,6867.542830,889.390804
min,26070.000000,283.000000
25%,26335.000000,686.000000
50%,26606.000000,1233.000000
75%,26859.000000,1942.000000
max,71862.000000,4888.000000


In [9]:
pts_df.head()

,aquamonitor_id,runoff_mm/yr
0,26070,532
1,26071,653
2,26072,501
3,26073,488
4,26074,503


In [10]:
# Check against my manual lookup above using ArcGIS
pts_df.query("aquamonitor_id == 26883")

,aquamonitor_id,runoff_mm/yr
774,26883,1869
